In [ ]:
using Oscar
using Combinatorics
pm = Polymake

In [ ]:
cd("..")

In [ ]:
currentDir = pwd()
include(joinpath(currentDir, "src/inputData38.jl"));
include(joinpath(currentDir, "src/fileHandling.jl"));
include(joinpath(currentDir, "src/tscCoordRing.jl"));
include(joinpath(currentDir, "src/matroidalSubd.jl"));
include(joinpath(currentDir, "src/Bmaximal.jl"));
include(joinpath(currentDir, "src/simplifyIdeal.jl"));
include(joinpath(currentDir, "src/specificMatroids.jl"));

The list ```G6``` contains all combinatorial types in $\mathsf{G}_6$. 

In [ ]:
G6Path = joinpath(currentDir,"groupsFinal/G6.dat")
G6 = file2SetVectors(G6Path);

In [ ]:
function tightSpanMaximalPolytopes(subd)
    TS = subd.pm_subdivision.TIGHT_SPAN; #print(1)
    MaxPoly = TS.MAXIMAL_POLYTOPES; #print(2);
    NMaxPoly = Int64(TS.N_MAXIMAL_POLYTOPES); #print(typeof(NMaxPoly)) 
    NVerts = Int64(TS.N_VERTICES); #print(typeof(NVerts));
    return [[j for j in 1:NVerts if  MaxPoly[i, j]] for i in 1:NMaxPoly]
end


# assumes that tight-span is 2-dimensional, not nec. pure
function countPolygonsInTightSpan(subd)
    TS = subd_o.pm_subdivision.TIGHT_SPAN; 
    D = Dict{Int64, Int64}(); #print(1)
    MaxPolyTS = tightSpanMaximalPolytopes(subd); #print(1)
    maxSides = maximum([length(P) for P in MaxPolyTS]); #print(maxSides)
    for i in 2:maxSides
        D[i] = length(findall([length(P) == i for P in MaxPolyTS]))
    end
    return D    
end

**Function**: ```nonLeaves(graphOscar::Graph)```

*Description*: Returns a ```Vector{Int64}``` listing the vertices that are not leaves of the graph ```graphOscar```. 


*Example*: 
```
```
---
**Function**: ```tightSpanNonLeaves(subd::SubdivisionOfPoints)```

*Description*: Returns the maximal cells of the tight span of ```subd``` that do not contain leaf vertices.


*Example*: 
```
w = [1,2,0,0,0,0,1,2,2,1,1,0,0,0,0,3,0,0,0,0,1,2,0,0,0,0,3,3,2,2,2,0,0,0,0,2,0,0,0,0,3,0,0,0,0,1,2,0,0,0,0,2,2,2,1,1]
subd = SubdivisionOfPoints(vDelta38[:,2:9], wt)
tightSpanNonLeaves(subd) == [[1, 2, 4, 10], [2, 3, 6, 7, 8, 10], [2, 10, 13, 15], [4, 5, 8, 9, 10], [4, 10, 11, 14, 15], [8, 10, 12, 15]]
```
---
**Function**: ```hasClaw(subd::SubdivisionOfPoints, Ms::Vector{Matroid})```

*Description*: Determines if the dual graph of ```subd``` has a tree consisting of a node connected to 4 other nodes, such that the matroid of the center node is isomorphic to $\mathsf{U}(12,34,56,78)$, and the matroids of the leaves are isomorphic to either $\mathsf{V}(12,34,56,7,8)$ or $\mathsf{W}(12,34,56,7,8)$. If this exists, the function returns a triple ```(m24, matsL1, matsL2)``` where ```m24``` is the center node, ```matsL1``` are the nodes whose matroids are isomorphic to $\mathsf{W}$, and ```matsL2``` are the nodes whose matroids are isomorphic to $\mathsf{V}$. If this does not exist, the function returns ```false```. 


*Example*: 
```
w = [1,2,0,0,0,0,1,2,2,1,1,0,0,0,0,3,0,0,0,0,1,2,0,0,0,0,3,3,2,2,2,0,0,0,0,2,0,0,0,0,3,0,0,0,0,1,2,0,0,0,0,2,2,2,1,1]
subd = SubdivisionOfPoints(vDelta38[:,2:9], w)
Ms = subd2Matroids(subd,3,8)
hasClaw(subd, Ms) == (10, [2, 4, 8, 15], [])
```
---
**Function**: ```intersectRestTS(TSMaxPoly::Vector{Vector{Int64}}, Fin::Vector{Int64})```

*Description*: Returns the vertices of ```Fin``` contained in some other maximal cell of the tight span.


*Example*: 
```
w = [1,2,0,0,0,0,1,2,2,1,1,0,0,0,0,3,0,0,0,0,1,2,0,0,0,0,3,3,2,2,2,0,0,0,0,2,0,0,0,0,3,0,0,0,0,1,2,0,0,0,0,2,2,2,1,1]
subd = SubdivisionOfPoints(vDelta38[:,2:9], w)
TSMaxPoly = tightSpanNonLeaves(subd)
intersectRestTS(TSMaxPoly, [2,3,6,7,8,10]) == [2,8,10]
```
---
**Function**: ```finsAttachedToClaw(subd::SubdivisionOfPoints, Ms::Vector{Matroid})```

*Description*: returns ```true``` if each (nonleaf) fin is attached to the tree $\mathsf{T}$ along exactly 2 edges, and ```false``` otherwise.  


*Example*: 
```
w = [1,2,0,0,0,0,1,2,2,1,1,0,0,0,0,3,0,0,0,0,1,2,0,0,0,0,3,3,2,2,2,0,0,0,0,2,0,0,0,0,3,0,0,0,0,1,2,0,0,0,0,2,2,2,1,1]
subd = SubdivisionOfPoints(vDelta38[:,2:9], w)
Ms = subd2Matroids(subd,3,8)
finsAttachedToClaw(subd,Ms) == true
```
---
**Function**: ```finIsBMaximal_G6(Ms::Vector{Matroid}, TSMaxPoly::Vector{Vector{Int64}}, Fin::Vector{Int64}, F::Ring, R::GradedPolynomialRing, x::)```

*Description*: Returns the list of bases ```mu``` of $\mathsf{Q}_{\mathsf{F}}$ (the matroid of the fin) such that the fin is $(B,\mu)$-maximal. 


*Example*: 
```
w = [1,2,0,0,0,0,1,2,2,1,1,0,0,0,0,3,0,0,0,0,1,2,0,0,0,0,3,3,2,2,2,0,0,0,0,2,0,0,0,0,3,0,0,0,0,1,2,0,0,0,0,2,2,2,1,1]
subd = SubdivisionOfPoints(vDelta38[:,2:9], w)
Ms = subd2Matroids(subd,3,8)
TSMaxPoly = tightSpanNonLeaves(subd)
R,x = makePolyRing(3,8,QQ)
finIsBMaximal_G6(Ms, TSMaxPoly, [2,3,6,7,8,10], QQ, R, x) == [[1, 2, 5], [1, 2, 6], [1, 4, 5], [1, 4, 6], [1, 5, 7], [1, 5, 8], [1, 6, 7], [1, 6, 8], [2, 3, 5], [2, 3, 6], [3, 4, 5], [3, 4, 6], [3, 5, 7], [3, 5, 8], [3, 6, 7], [3, 6, 8]]

```
---
**Function**: ```allFinsBMaximal_G6(Ms::Vector{Matroid}, TSMaxPoly::Vector{Vector{Int64}}, F::Ring, R::GradedPolynomialRing, x::)```

*Description*: Returns ```true``` if each fin is $B$-maximal, and ```false``` otherwise. 


*Example*: 
```
w = [1,2,0,0,0,0,1,2,2,1,1,0,0,0,0,3,0,0,0,0,1,2,0,0,0,0,3,3,2,2,2,0,0,0,0,2,0,0,0,0,3,0,0,0,0,1,2,0,0,0,0,2,2,2,1,1]
subd = SubdivisionOfPoints(vDelta38[:,2:9], w)
Ms = subd2Matroids(subd,3,8)
TSMaxPoly = tightSpanNonLeaves(subd)
R,x = makePolyRing(3,8,QQ)
finIsBMaximal_G6(Ms, TSMaxPoly, QQ, R, x) == true
```


In [ ]:
matroid2by4 = matroidU([1,2], [3,4], [5,6], [7,8])
matroidLeaf1 = matroidV([1,2],[3,4],[5,6],[7],[8])
matroidLeaf2 = matroidW([1,2],[3,4],[5,6],[7],[8])


function nonLeaves(graphOscar)
    notLeaves = [v for v in 1:Graphs.nv(graphOscar) if length(Graphs.all_neighbors(graphOscar, v)) > 1 ]
    return notLeaves
end



function tightSpanNonLeaves(subd)
    Gra = subd.pm_subdivision.POLYHEDRAL_COMPLEX.DUAL_GRAPH
    graphOscar = Graphs.Graph{Graphs.Undirected}(Gra.ADJACENCY)
    Ls = leavesGraph(graphOscar)
    TS = tightSpanMaximalPolytopes(subd)
    
    TS_noLeaves = [F for F in TS if length(intersect(F,Ls)) == 0]
    
    return sort!(TS_noLeaves)
end



function hasClaw(subd, Ms)
    mats2by4 = [i for i in 1:length(Ms) if is_isomorphic(matroid2by4, Ms[i])]
    matsL1 = [i for i in 1:length(Ms) if is_isomorphic(matroidLeaf1, Ms[i])]
    matsL2 = [i for i in 1:length(Ms) if is_isomorphic(matroidLeaf2, Ms[i])]
        
    if !(length(mats2by4) ==1 && length(matsL1) + length(matsL2) == 4)
        return false
    end
    m24 = mats2by4[1]
    
    Gra = subd.pm_subdivision.POLYHEDRAL_COMPLEX.DUAL_GRAPH
    graphOscar = Graphs.Graph{Graphs.Undirected}(Gra.ADJACENCY)
    
    nbsM24 = [i for i in Graphs.all_neighbors(graphOscar, m24) if (i in matsL1 || i in matsL2)]
    
    if Set(nbsM24) == union(Set(matsL1), Set(matsL2))
        return (m24, matsL1, matsL2)
    else
        return false
    end
end

function intersectRestTS(TSMaxPoly, Fin)
    intFs = [intersect(Fin, G) for G in TSMaxPoly if G != Fin]
    return sort!(unique!(vcat(intFs...)))
end

function finsAttachedToClaw(subd, Ms)
    claw = hasClaw(subd, Ms)
    if typeof(claw) == Bool
        if !claw
            return false
        end
    end  
    
    (m24, matsL1, matsL2) = claw
        
    vertsClaw = unique!(vcat([m24], matsL1, matsL2))
    TSMaxPoly = tightSpanNonLeaves(subd)
    
    FsIntR = [intersectRestTS(TSMaxPoly, F) for F in TSMaxPoly]
    
    return all([(length(C) == 3 && issubset(C, vertsClaw) && m24 in C) for C in FsIntR])
        
end


function finIsBMaximal_G6(Ms, TSMaxPoly, Fin, F, R, x)
    connectingPath = intersectRestTS(TSMaxPoly, Fin)
    Ms_Exterior = [Ms[i] for i in Fin if i ∉ connectingPath]
    Ms_Fin = [Ms[i] for i in Fin]
    Bs = commonBasis(Ms_Fin)
    return existsBasisBMaximal_Ms(Ms_Exterior, F, R, x, Bs)
end

function allFinsBMaximal_G6(Ms, TSMaxPoly, F, R, x)
    BMaxBases = [finIsBMaximal_G6(Ms, TSMaxPoly, Fin, F, R, x) for Fin in TSMaxPoly]
    return all([length(Bs) > 0 for Bs in BMaxBases])
end

We check that, for each $\mathsf{w}$ in $\mathsf{G}_6$, the tight span $\mathsf{TS}(\mathsf{w})$ contains a star-tree $\mathsf{T}$ that has 4 leaves, such that the matroid of the center node is isomorphic to $\mathsf{U}(12,34,56,78)$, and the matroids of the 4 leaves are isomorphic to either $\mathsf{V}(12,34,56;7,8)$ or $\mathsf{W}(12;34,56;7,8)$. Furthermore, we verify that there are exactly 6 fins of $\Sigma_L \subset \mathsf{TS}(\mathsf{w})$, with one connected along each pair of edges of $\mathsf{T}$. Finally we check that each of these fins is $B$-maximal, and therefore the limit $\operatorname{Gr}(\mathsf{w})$ is smooth and irreducible.

In [ ]:
passes_test = []
fails_test = []

R,x = makePolyRing(3,8,QQ)

for w in G6
    subd = SubdivisionOfPoints(vDelta38[:,2:9], -w); 
    TS = tightSpanNonLeaves(subd)
    Ms = subd2Matroids(subd, 3, 8);
    R, x = makePolyRing(3, 8, QQ);
    
    if !finsAttachedToClaw(subd, Ms)
        push!(fails_test, w)
    elseif allFinsBMaximal_G6(Ms, TS, QQ, R, x)
        push!(passes_test, w)
    else
        push!(fails_test, w)
    end
end

In [ ]:
length(fails_test) == 0

## Dimension of $\operatorname{Gr}(\mathsf{w})$

Next we check that the dimensions $\operatorname{Gr}(\mathsf{w})$ are 15 for each $\mathsf{w}$ in $\mathsf{G}_6$. We do this by applying Proposition 6.16, which is implemented in the function ```dimLimitG6``` below. 

In [ ]:
function minBDim(M)
    DM = basis2DimB(M)
    return min(values(DM)...)
end

function matroidOfEdge(Ms,e,n)
    B1 = bases(Ms[e[1]])
    B2 = bases(Ms[e[2]])
    return matroid_from_bases(intersect(B1,B2), n)
end


function dimLimitG6(w,F,d,n,R,x)
    subd = SubdivisionOfPoints(vDelta38[:,2:n+1], -w); 
    Ms = subd2Matroids(subd,d,n)
    claw = hasClaw(subd, Ms)
    dimClaw = 11 + length(claw[2])
    
    (m24, matsL1, matsL2) = claw
    
    vertsClaw = unique!(vcat([m24], matsL1, matsL2))
    
    Gr = subd.pm_subdivision.POLYHEDRAL_COMPLEX.DUAL_GRAPH
    oG = Graphs.Graph{Graphs.Undirected}(Gr.ADJACENCY)
    LeavesVerts = leavesGraph(oG)
    TS = tightSpanMaximalPolytopes(subd)
    
    Fins = [F for F in TS if length(intersect(F,LeavesVerts)) == 0]
    LeavesEdges = [L for L in TS if length(intersect(L,LeavesVerts)) != 0]
    
    leafVMatroids = [Ms[i] for i in LeavesVerts]
    leafEMatroids = [matroidOfEdge(Ms,L,n) for L in LeavesEdges]
    
    FinsExposedV = union!([[i for i in Fin if i ∉ vertsClaw] for Fin in Fins]...)
    FinsExposedE = union!([[sort!([i,j]) for j in Graphs.all_neighbors(oG, i) if j ∉ LeavesVerts] for i in FinsExposedV]...)
        
    
    FinsExposedVMats = [Ms[i] for i in FinsExposedV]
    FinsExposedEMats = [matroidOfEdge(Ms,e,n) for e in FinsExposedE]
    FinsBases = [ intersect([bases(Ms[i]) for i in Fin]...)  for Fin in Fins]
    FinsMats = [matroid_from_bases(Bs, n) for Bs in FinsBases]
    
    dimsLeavesV = Vector{Int64}([dimTSC_Optimized(M, F, R, x) for M in leafVMatroids ]); 
    dimsLeavesE = Vector{Int64}([dimTSC_Optimized(M, F, R, x) for M in leafEMatroids ]); 
    
    dimsFinsV = Vector{Int64}([dimTSC_Optimized(M, F, R, x) for M in FinsExposedVMats ]); 
    dimsFinsE = Vector{Int64}([dimTSC_Optimized(M, F, R, x) for M in FinsExposedEMats ]); 
    dimsFinsF = Vector{Int64}([dimTSC_Optimized(M, F, R, x) for M in FinsMats ]);
    
    
    return sum(dimsLeavesV) - sum(dimsLeavesE) + sum(dimsFinsV) - sum(dimsFinsE) + sum(dimsFinsF) + dimClaw
    
end

In [ ]:
R, x =  makePolyRing(3,8,QQ)
dims_G6 = []

dimFile = joinpath(currentDir, "G6Data/dims_G6.dat")
io1 = open(dimFile, "w") 
close(io1)

for i in 1:length(G6)    # change range for testing
    w = G6[i]
    d = dimLimitG6(w, QQ, 3, 8, R, x)
    push!(dims_G6, d)
    
    open(dimFile, "a") do io
        write(io, string("dim w", i, " = " , d, "\n"))
        end
    
end


In [ ]:
all([d == 15 for d in dims_G6])

Here is Example 6.20 in the paper.

In [ ]:
subd = SubdivisionOfPoints(vDelta38[:,2:9], -G6[5]); 
w = [2,2,0,0,0,0,2,2,2,4,3,0,0,0,0,1,0,0,0,0,1,2,0,0,0,0,1,1,1,1,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,1,2,2,2,2]
Vector{Int64}(subd.pm_subdivision.MIN_WEIGHTS) == w

This next cell verifies the characterization of $\mathsf{w}$ in the paper.

In [ ]:
d3n8 = collect(powerset([1:8;], 3, 3))
wDict = [(d3n8[i], w[i]) for i in 1:56 if w[i] != 0]

In [ ]:
GammaPolymake = subd.pm_subdivision.POLYHEDRAL_COMPLEX.DUAL_GRAPH
Gamma = Graphs.Graph{Graphs.Undirected}(GammaPolymake.ADJACENCY)
Ms = subd2Matroids(subd,3,8);

Next, we verify that the matroids in the paper correct. 

In [ ]:
M1 = matroidW([5,6,7,8],[1],[3],[2],[4])
M2 = matroidW([7,8],[1],[3],[2,4],[5,6])
M3 = matroidU([1],[2,4,5,6,8],[3],[7])
M4 = matroidW([7],[1],[3],[2,4,5,6],[8])
M5 = matroidU([1,3],[2],[4],[5,6,7,8])
M6 = matroidW([1,3,7,8],[2],[4],[5],[6])
M7 = matroidW([7,8],[1,3],[2,4],[5],[6])
M8 = matroidW([2,4,7,8],[1],[3],[5],[6])
M9 = matroidU([1,3],[2,4],[5,6],[7,8])
M10 = matroidW([1,3],[2],[4],[5,6],[7,8])
M11 = matroidW([1,3,5,6],[2],[4],[7],[8])
M12 = matroidU([1,2,3,4],[5,6],[7],[8])
M13 = matroidU([1,2,3,4],[5],[6],[7,8])
M14 = matroidW([1,2,3,4],[5],[6],[7],[8])
M15 = matroidW([5,6],[1,3],[2,4],[7],[8])

all([bases(Ms[1]) == bases(M1), 
     bases(Ms[2]) == bases(M2), 
     bases(Ms[3]) == bases(M3), 
     bases(Ms[4]) == bases(M4),
     bases(Ms[5]) == bases(M5), 
     bases(Ms[6]) == bases(M6),
     bases(Ms[7]) == bases(M7), 
     bases(Ms[8]) == bases(M8), 
     bases(Ms[9]) == bases(M9), 
     bases(Ms[10]) == bases(M10),
     bases(Ms[11]) == bases(M11), 
     bases(Ms[12]) == bases(M12),
     bases(Ms[13]) == bases(M13),
     bases(Ms[14]) == bases(M14), 
     bases(Ms[15]) == bases(M15)
        ])

Run the cell below to see the dual graph of the subdivision. 

In [ ]:
pm.visual(GammaPolymake)